In [3]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
import json
import re
import time
import sys
import os
import datetime

def counter_for_files(path):# Нахождение файлов в директории
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [69]:
def finding_popular(most_popular):# Обработка 5% наиболее популярных сущностей
    if len(most_popular) * 0.05 < 10:
        return list(most_popular.iloc[:10].entity)
    elif len(most_popular) * 0.05 >= 100:
        return list(most_popular.iloc[:100].entity)
    else:
        return list(most_popular.iloc[:int(len(most_popular) * 0.05)].entity)
    
def aggregation(name):
    stats_df = pd.read_csv(f'analyzed_entities_{name}.csv',encoding='utf8',sep=';')
    most_popular = pd.read_csv(f'entities_{name}_with_popularity.csv',encoding='utf8',sep=';')
    length = len(most_popular)
    most_popular = finding_popular(most_popular)
    print(len(most_popular))
    print(length)
    def filter_func(x):
        return x['entity_id'].max() in most_popular
    
    popular_stats = stats_df.groupby('entity_id').filter(filter_func).groupby('prop_name').sum().sort_values(by = ['prop_count','prop_count_value'],ascending = False)
    popular_stats['prop_counter'] = popular_stats['prop_count']/len(most_popular) * 100
    popular_stats.to_csv(f'entities_{name}_aggregated_with_popularity.csv', encoding='utf-8-sig',sep = ';')
    
    stats_aggregated = stats_df.groupby('prop_name').sum().sort_values(by = ['prop_count','prop_count_value'],ascending = False)
    stats_aggregated['prop_counter'] = stats_aggregated['prop_count']/length * 100
    stats_aggregated.to_csv(f'entities_{name}_aggregated.csv', encoding='utf-8-sig',sep = ';')
    


In [70]:
name = counter_for_files(os.getcwd())[0].split('\\')[-1].split('.')[0].split('_')[-2:]
name = '_'.join(name)
aggregation(name)

11
229


In [68]:
stats_df = pd.read_csv(f'analyzed_entities_country_Q6256.csv',encoding='utf8',sep=';')
sset = set(['from' , 'to'])
for k,v in stats_df.iterrows():
    if list(stats_df[(stats_df.prop_dir == (sset - set([v['prop_dir']])).pop()) & (stats_df.entity_id == v.entity_id) & (stats_df.prop_name == v.prop_name)].prop_name) != []:
        ind = list(stats_df[stats_df.prop_dir == (sset - set([v['prop_dir']])).pop()].index)[0]
        stats_df.loc[ind , 'prop_count_value'] += v.prop_count_value
        stats_df.drop(k, inplace=True)
        
# stats_df['prop_count'] = 1
stats_df.to_csv(f'analyzed_entities_country_Q6256.csv',encoding='utf-8-sig',sep=';' , index = False)

most_popular = pd.read_csv(f'entities_country_Q6256_with_popularity.csv',encoding='utf8',sep=';')
most_popular.sort_values(by = ['popularity'],ascending = False).to_csv(f'entities_country_Q6256_with_popularity.csv',encoding='utf-8-sig',sep=';', index = False)
